In [6]:
import torch
ckpt = torch.load("../checkpoints/best_multimodal.pt", map_location="cpu")
w = ckpt["model_state"]["fusion.0.weight"]
A = 128
w_a, w_s = w[:, :A], w[:, A:]
print("||w_audio||", w_a.norm().item(), "||w_sensor||", w_s.norm().item())


||w_audio|| 6.510303974151611 ||w_sensor|| 6.426955699920654
